In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
ls expts/rel3_tsel_ref_dial_model_separate/directions

bad_filtered/
base_512_bsz-64_lr-5e-3_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_bsz-64_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm-lPm-lPpm_prb-lpm-lm-lPm-lPpm/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm_prb-lpm-lm/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_2/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_3/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-fil

In [5]:
ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

 1_best.stdict
 1_best.th
 1_ep-30.stdict
 1_ep-30.th
'eval_dev_cand=40_oracle.out'
'eval_dev_cand=40_speaker-weight=0.1.out'
'eval_dev_cand=40_speaker-weight=0.2.out'
'eval_dev_cand=40_speaker-weight=0.3.out'
'eval_dev_cand=40_speaker-weight=0.4.out'
'eval_dev_cand=40_speaker-weight=0.5.out'
'eval_dev_cand=40_speaker-weight=0.6.out'
'eval_dev_cand=40_speaker-weight=0.7.out'
'eval_dev_cand=40_speaker-weight=0.8.out'
'eval_dev_cand=40_speaker-weight=0.9.out'
'eval_dev_cand=40_speaker-weight=0.out'
'eval_dev_cand=40_speaker-weight=1.0.out'
 eval_dev_l0.out
 eval_dev_l0_ref_annotations.json
 train_fold-1.out


In [6]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
    'encoder-filtered-separate-diffs': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/1_ep-30.th',
}

In [7]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 10
vocab size: 649
dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [8]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [9]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [10]:
len(scenarios1)

9000

In [11]:
len(scenarios2)

4500

In [12]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [13]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [14]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [15]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [16]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [17]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [18]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 796.53it/s]


In [19]:
validset_stats

{'n': 74030, 'self_n': 37015, 'nonpadn': 69348, 'self_nonpadn': 34674}

In [20]:
import torch

In [21]:
from agent import RnnAgent

In [22]:
from selfplay import make_parser

In [23]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [24]:
def is_selection(out):
    return '<selection>' in out

In [25]:
# display

In [26]:
from dialog import DialogLogger

In [27]:
from IPython.display import SVG, display, HTML

In [28]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [29]:
SVG_SCALE=0.4

In [30]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [44]:
def display_attns(scenario, attns, agent_id):
    svgs = [DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id]]
#     if name is not None:
#         print("{}: {}".format(name, attn))
    for attn in attns:
        attn = attn.flatten().detach().numpy()
        svgs.append(DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE))
    display_svgs(svgs)

In [31]:
# training emulation

In [32]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:11<00:00, 706.89it/s]


In [33]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [34]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [35]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [36]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate', 'encoder-filtered-separate-diffs'])

In [38]:
model = models['encoder-filtered-separate-diffs']

In [39]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [45]:
def force_contexts_and_mentions(index, datasplit=validset, samples=10, sample_temperature=0.25, argmax_temperature=0.005):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    scenario = all_scenarios[scenario_id]
    agent = alice
    agent.feed_context(ctx.flatten())
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    writer = alice
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]

        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
            mentions = [dots_mentioned_per_ref[:,mention_ix].float()  for mention_ix in range(dots_mentioned_per_ref.size(1))]
            display_attns(scenario, mentions, writer.agent_id)
#             for mention_ix in range(dots_mentioned_per_ref.size(1)):
#                 display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            for sample_ix in range(samples):
                pred_outs, _, _, _, extra = writer.model.write(
                    writer.ctx, writer.ctx_differences, writer.ctx_h, writer.reader_and_writer_lang_h, 
                    words_left, sample_temperature,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables
                )
                print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
            pred_outs, _, _, _, extra = writer.model.write(
                writer.ctx, writer.ctx_differences, writer.ctx_h, writer.reader_and_writer_lang_h, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('argmax\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

            out = writer.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
            )

    #         if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
    #             display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')

    #         if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
    #             display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')

    #         for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
    #             display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))

        else:
            out = words_og
            writer.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables)

        print('human\t{}'.format(' '.join(out)))
        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out) or words_left <= 1:
            break
        sentence_ix += 1

In [46]:
force_contexts_and_mentions(1)

human	THEM: i have one large very light dot way off on its own <eos>

----------------------------------------



sample 0	i see a triangle of light grey dots , one is slightly lighter and slightly smaller <eos>
sample 1	i have a triangle of light grey dots , the top one is a little bigger and lighter <eos>
sample 2	i have a cluster of three dots . the top one is the lightest and the smallest <eos>
sample 3	i have a triangle of dots , the top one is the lightest and the smallest <eos>
sample 4	i see a cluster of three dots that form a triangle . the top one is the lightest and the smallest <eos>
sample 5	i have a cluster of three dots . the middle one is the lightest and smallest <eos>
sample 6	i see a group of three dots . the top one is the lightest <eos>
sample 7	i have three dots in a triangle , the top one is the lightest <eos>
sample 8	i have a triangle of light gray dots , the top one is a little darker and slightly smaller <eos>
sample 9	i have a cluster of 3 dots , the bottom one is the lightest and the smallest <eos>
argmax	i have a triangle of dots , the top one is the lightest and the 

In [47]:
force_contexts_and_mentions(2)

human	THEM: i have to two same tiny and light dot close in vertical <unk> <eos>

----------------------------------------



sample 0	i have a pair of light grey dots , one is slightly lower and to the left <eos>
sample 1	i have a pair of small dots , the one on the right is slightly higher and slightly lighter <eos>
sample 2	i have a triangle of dots that are the same size and color , and the one on the right is a little lighter and a little bit to the right <eos>
sample 3	i see two small light gray dots , they are both light grey <eos>
sample 4	i have three dots that are very close together and the top one is a little larger and slightly lighter <eos>
sample 5	i see that . let 's pick the top one . <eos>
sample 6	i have two small light grey dots , one is slightly lower and to the right <eos>
sample 7	i have a pair of small dots , the one on the right is slightly lower and slightly lighter <eos>
sample 8	i see two very light grey dots , one is slightly lower and to the right <eos>
sample 9	i have a pair of light grey dots , one on the right is slightly lighter <eos>
argmax	i have two small light grey dots ,

In [48]:
force_contexts_and_mentions(3)

sample 0	i have a large black dot by itself <eos>
sample 1	i have a large black dot by itself <eos>
sample 2	i have one large black dot by itself <eos>
sample 3	i have a large black dot by itself <eos>
sample 4	i have one large black dot by itself <eos>
sample 5	i have a large black dot by itself <eos>
sample 6	i have one large black dot . it is large <eos>
sample 7	i have a large black dot by itself <eos>
sample 8	i have one large black dot , it is near the upper left of my circle . <eos>
sample 9	i have a large black dot by itself <eos>
argmax	i have a large black dot by itself <eos>
human	do you see one very large black dot ? it 's the only dark dark one <eos>

----------------------------------------

human	THEM: i see that but i also have a medium sized very dark one as well . is the large black one almost exactly in between two other dots . the one on the right being lightly smaller and very light and the one on the left being smaller and medium grey ? <eos>

--------------------

sample 0	yes ! let 's pick the black one . <selection>
sample 1	yes , let 's choose the black one . <selection>
sample 2	yes , let 's choose the large black one . <selection>
sample 3	yes , let 's pick the black one . <selection>
sample 4	yes ! let 's pick the large black one ! <selection>
sample 5	yes . let 's pick the large black one <selection>
sample 6	yes . let 's choose the large black one . <selection>
sample 7	yes . let 's pick the large black one . <selection>
sample 8	yes , let 's pick the large black one . <selection>
sample 9	yes , let 's pick the black one . <selection>
argmax	yes , let 's pick the large black one . <selection>
human	yes ... shall we choose the larger black spot then ? <eos>

----------------------------------------

human	THEM: yep <selection>

----------------------------------------



In [49]:
force_contexts_and_mentions(4)

human	THEM: i have a tiny dark dot with a large dark dot directly above it <eos>

----------------------------------------



sample 0	i have that . is there a smaller dark grey dot to the left of it ? <eos>
sample 1	i have that too . is there a small black dot to the right of it ? <eos>
sample 2	i have that . is the one on the right slightly smaller than the one on the right ? <eos>
sample 3	i have that . is there a small black dot to the right of it ? <eos>
sample 4	i have that . is it above a small black dot ? <eos>
sample 5	i have that . is there a small black dot to the right of it ? <eos>
sample 6	i have that too . is there a small black dot to the right of it ? <eos>
sample 7	i have that too . is there a small black dot to the left of it ? <eos>
sample 8	i have that , i also have a medium black dot with a larger black dot above and to the left of it <eos>
sample 9	i have that . is there a small black dot to the left of it ? <eos>
argmax	i have that . is there a small black dot to the right of it ? <eos>
human	i have a line of three dots , they are all quite dark . the middle is the smallest <eos>

----

sample 0	yes , the small one is to the right , but the smallest is the darkest <eos>
sample 1	yes , lets choose the small one , i think that is the one <selection>
sample 2	yes , i have that one too . let 's choose that one . <selection>
sample 3	yes , i think i see it . let 's pick the middle one . <selection>
sample 4	yes , let 's choose the small one . the top one . <selection>
sample 5	yes , i think we should pick the one that is the darkest one <selection>
sample 6	yes . let 's pick the small dark one , i think that is the same one <selection>
sample 7	yes , let 's pick the small one on the bottom , the top one . <selection>
sample 8	yes , i have that one . let 's pick the tiny one . <selection>
sample 9	yes , the smaller one is the darkest ? if so click it ! <eos>
argmax	yes , i think we should pick the one that is the darkest <selection>
human	yes . i see your small dot too ... shall we select that tiny dot ? <eos>

----------------------------------------

human	THEM: ok <eos>


In [50]:
force_contexts_and_mentions(5)

human	THEM: hi ! do you have three dots in a line ? the one to the left is the medium grey , the one in the middle is light and the last one is darkest <eos>

----------------------------------------

sample 0	yes <eos>
sample 1	yes <eos>
sample 2	yes <eos>
sample 3	yes <eos>
sample 4	yes <eos>
sample 5	yes <eos>
sample 6	yes ! <eos>
sample 7	yes <eos>
sample 8	yes <eos>
sample 9	yes <eos>
argmax	yes <eos>
human	yes <eos>

----------------------------------------

human	THEM: okay , select the lightest one ! <selection>

----------------------------------------



In [51]:
force_contexts_and_mentions(6)

human	THEM: group of four dots that look like a backwards 7 <eos>

----------------------------------------



sample 0	i have a large black dot on the right side of the circle <eos>
sample 1	yes , choose the darkest one ? <eos>
sample 2	yes , is the middle one the darkest ? <eos>
sample 3	is the top one black ? <eos>
sample 4	yes , let 's pick the largest one that is the darkest <eos>
sample 5	yes , i have that one . <eos>
sample 6	yes , the middle one is dark and large ? <eos>
sample 7	yes , lets choose the darkest one <eos>
sample 8	yes , pick the darkest one on the right <selection>
sample 9	yes , is the middle one the darkest ? <eos>
argmax	yes , lets choose the darkest one on the right <selection>
human	yes . select the large black one ? <eos>

----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [52]:
force_contexts_and_mentions(7)

human	THEM: large medium grey one alone <eos>

----------------------------------------



sample 0	yes , i see that one . <eos>
sample 1	yes , i see that one . <eos>
sample 2	yes , i have that one . <eos>
sample 3	yes , i see that one . <eos>
sample 4	yes , it is on the right side of the circle <eos>
sample 5	yes , i see that one <eos>
sample 6	yes , i have that one . <eos>
sample 7	yes , i see that one . <eos>
sample 8	yes , i have a large light grey dot <eos>
sample 9	yes i see that one <eos>
argmax	yes , i have that one . <eos>
human	medium light grey one alone <eos>

----------------------------------------

human	THEM: above the rest ? <eos>

----------------------------------------

sample 0	yes <eos>
sample 1	yes <eos>
sample 2	yes <eos>
sample 3	yes <eos>
sample 4	yes <eos>
sample 5	yes <eos>
sample 6	yes <eos>
sample 7	yes <eos>
sample 8	yes <eos>
sample 9	yes <eos>
argmax	yes <eos>
human	in the top left corner yes <eos>

----------------------------------------

human	THEM: click that one <selection>

----------------------------------------



In [53]:
force_contexts_and_mentions(8)

sample 0	i have two dots very close together . the one on the right is slightly smaller and darker . <eos>
sample 1	two dots very close together ? one on top is darker and smaller <eos>
sample 2	i see two dots very close together . the one on the right is slightly smaller , darker and slightly to the left . <eos>
sample 3	i have two dots very close together . the one on the right is slightly smaller and darker . <eos>
sample 4	i have two dots very close together . the one on the left is slightly smaller and darker . <eos>
sample 5	i see two dots very close together . the one on top is smaller , darker and slightly to the right . <eos>
sample 6	i have two dots very close together . the one on top is slightly smaller and darker . <eos>
sample 7	i have two dots very close together . the one on the left is slightly smaller and darker . <eos>
sample 8	i have two dots very close together . the one on the right is slightly smaller and darker . <eos>
sample 9	i see two dots very close together

sample 0	i do n't see that . <eos>
sample 1	i do n't have that . <eos>
sample 2	i do n't see that <eos>
sample 3	i do n't see that <eos>
sample 4	i do n't have that <eos>
sample 5	i do n't see that <eos>
sample 6	i do n't see that <eos>
sample 7	i do n't have that <eos>
sample 8	i do n't see that <eos>
sample 9	i do n't have that <eos>
argmax	i do n't have that <eos>
human	i do n't have that <eos>

----------------------------------------

human	THEM: okay , back to your original dots . they are both large and far apart . pale one , and then at 5 o clock position another one , about 2 inches ( <unk> ) away ? is that it ? <eos>

----------------------------------------



sample 0	yes , the two dots are on the right , one is darker than the other <eos>
sample 1	yes , i have that too . the top one is darker than the other two ? <eos>
sample 2	no . i have a triangle of three dots , the top one is dark , the bottom one is light grey <eos>
sample 3	i have two dots that are the same size . the one on the left is slightly darker , the one on the right is light grey <eos>
sample 4	no , i have a pair of dots . the one on the right is darker and the one on the right is light grey <eos>
sample 5	no . i have two dots very close together . the one on the left is darker and smaller than the one on the left . <eos>
sample 6	yes , i have that . the one on the right is a little darker than the one on the right <eos>
sample 7	no , i have two small dots . one is dark grey and one is light gray . <eos>
sample 8	no . i have two dots that are very close together . the one on the left is slightly darker and darker than the one on the right . <eos>
sample 9	i have a pair of d

sample 0	yes , let 's pick the light one . <eos>
sample 1	yes , let 's choose the light one on the left <eos>
sample 2	yes , lets pick the light one <selection>
sample 3	yes , lets pick the light one <eos>
sample 4	yes , let 's pick the light one . <selection>
sample 5	yes . let 's choose the light one . <selection>
sample 6	yes . let 's choose the light one . <selection>
sample 7	yes , lets pick the light one <eos>
sample 8	yes , let 's pick the lighter one . <selection>
sample 9	yes , let 's pick the light one . <selection>
argmax	yes , let 's pick the light one . <selection>
human	yes , let 's choose the pale one <eos>

----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [54]:
force_contexts_and_mentions(9)

human	THEM: do you see one very large black dot ? it 's the only dark dark one <eos>

----------------------------------------



sample 0	i have a large black dot , and to the right of it is a smaller light grey dot , and to the right of it is a smaller light grey dot . i have a cluster of 3 dots <eos>
sample 1	i have a large black dot . above and to the right is a smaller grey dot , and below that black dot is a medium gray dot and to the right of it is a small light dot <eos>
sample 2	i have a large black dot , and to the right of it is a small light dot , and below that black dot is a medium grey dot . <eos>
sample 3	i have a large black dot . above it and to the right is a small light grey dot , and to the left of it is a smaller black dot . below that black dot is a light grey dot <eos>
sample 4	i have a large black dot . above and to the left is a small light grey dot , below and to the right of it is a small light dot , and above that medium grey dot is a medium gray dot <eos>
sample 5	i have a large black dot . below it and to the right is a small light grey dot , and to the right of it is a smaller dark

In [55]:
force_contexts_and_mentions(10)

sample 0	two tiny dots close to each other , one on the right is slightly darker and slightly larger <eos>
sample 1	i have two small dots very close to each other . the one on the right is slightly darker and slightly smaller . <eos>
sample 2	i have two tiny dots close to each other , one is slightly darker and slightly lower <eos>
sample 3	two tiny dots almost vertical to each other ? one on the right is darker and slightly larger <eos>
sample 4	i have two small dots almost on a vertical line . the one on the left is slightly darker and slightly larger . <eos>
sample 5	i have two small dots very close to each other . the one on the right is slightly darker and a little darker <eos>
sample 6	i have two tiny dots close to each other . the one on top is slightly to the right and slightly darker <eos>
sample 7	two small dots , one is slightly darker and to the right of the other ? <eos>
sample 8	i have two small dots close to each other . the one on the right is slightly darker , and a li

sample 0	ok , let 's do it ! <selection>
sample 1	ok , lets click the tiny one <selection>
sample 2	ok , lets click the smaller one <selection>
sample 3	ok , the one on the right is a little darker . <eos>
sample 4	ok , let 's do the one on the right . <selection>
sample 5	ok , lets pick the top one <selection>
sample 6	ok , lets click the top one <selection>
sample 7	ok , lets click the top one <selection>
sample 8	ok , lets click the top one ? <eos>
sample 9	ok , the one on the right is slightly darker . <eos>
argmax	ok , lets click the smaller one <selection>
human	okay i will click the top one . <selection>

----------------------------------------



In [56]:
force_contexts_and_mentions(11)

sample 0	i see two dots very close to each other . <eos>
sample 1	i have two dots of same size and color dots close to each other <eos>
sample 2	two dots very close together and diagonal to each other ? <eos>
sample 3	i have two dots very close to each other , same size <eos>
sample 4	i have two dark dots close to each other <eos>
sample 5	i have two dark dots close to each other <eos>
sample 6	i have two dots that are very close together . they are both medium size <eos>
sample 7	i have two medium sized dots very close to each other <eos>
sample 8	i have two dots of similar size and color dots close to each other <eos>
sample 9	i have two dots very close to each other , very close together <eos>
argmax	i have two dots very close to each other <eos>
human	two medium sized dots really close to each other ? <eos>

----------------------------------------

human	THEM: i have many black dot , but i have one set of medium grey the one on the right is slightly larger and darker ? <eos>

----

sample 0	yes i see that . choose the smaller one <selection>
sample 1	yes i see that . pick the smaller one <selection>
sample 2	yes i have that . let 's pick the smaller one <eos>
sample 3	yes i see that . choose the smaller one <selection>
sample 4	yes , i have that . let 's pick the smaller one <selection>
sample 5	yes i see that . pick the smaller one <selection>
sample 6	yes i see that . choose the smaller one <selection>
sample 7	yes i see that . pick the smaller one <selection>
sample 8	yes i see that . choose the smaller one <selection>
sample 9	yes i see that . choose the smaller one <selection>
argmax	yes i see that . choose the smaller one <selection>
human	i have that ... the left one is a little higher up . <eos>

----------------------------------------

human	THEM: yes . let 's choose that one ... the smaller one on the left . = ) <selection>

----------------------------------------



In [72]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--
